In [6]:
from evaluation_copilot.base import EvaluationCopilot, ImprovementCopilot
from evaluation_copilot.models import EvaluationInput, ImprovementInput
from dotenv import load_dotenv
import openai
from anthropic import Anthropic
import os

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai_client = openai.Client()
anthropic_client = Anthropic(
    api_key=os.environ["ANTHROPIC_API_KEY"],
)

In [2]:
transcripts_dir = 'transcripts'
files = os.listdir(transcripts_dir)
files.sort()
with open(os.path.join(transcripts_dir, files[0]), 'r') as file:
    EPISODE_TRANSCRIPT = file.read()

In [3]:
message = anthropic_client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1024,
    temperature=0,
    system="You are good at extracting story arcs from transcript of TV episodes. You will be a given a transcript of an episode. Your task is to extract the main story arcs from this episode. A story arc is a narrative thread that runs through part or all of the episode, involving certain characters and exploring certain themes. An arc has a beginning, middle and end. It will often center around a conflict or challenge that the characters need to overcome. \n\nFor example, in an episode of Friends, one story arc might be:\n- Joey and Chandler centers around a humorous fight over a chair. The conflict begins when Chandler leaves the room for a moment and Joey takes his seat. When Chandler returns and demands the seat back, Joey refuses, leading to a series of tit-for-tat pranks between the two. The situation escalates comically when Joey decides to make a point about not sharing by wearing all of Chandler's clothes at once, declaring, \"Look at me, I'm Chandler. Could I BE wearing any more clothes?\" This is done as a form of exaggerated revenge for Chandler hiding Joey's underwear, and it leads to a memorable and funny moment involving physical comedy and slapstick humor.\n\nPlease carefully read through the transcript and identify the main story arcs. For each one, think about:\n<scratchpad>\n- What is the key conflict or challenge driving this arc? How is it set up and how is it resolved?\n- Which characters are most central to this arc? What role do they each play?\n- What deeper themes, if any, are explored through this storyline? \n</scratchpad>\n\nThen, output the story arcs in the following JSON format:\n\n<example>\n[\n  {\n    \"label\": \"Joey and Chandler's Fight\",\n    \"description\": \"Joey and Chandler bicker over a chair leading to escalating pranks including Joey wearing all of Chandler's clothes as an act of revenge.\",\n    \"characters\": [\"Joey\", \"Chandler\"],\n    \"themes\": [\"Friendship\", \"Conflict\", \"Petty revenge\"]\n  },\n  ...\n]\n</example>\n\n \n\nMake sure the story arc objects are printed in valid JSON format. Only include the valid JSON and do not include any other text. I just want the JSON from you.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"\n\n<episode_transcript>\n{EPISODE_TRANSCRIPT}\n</episode_transcript>\n\n"
                }
            ]
        }
    ]
).content[0].text
print(message)

Here are the main story arcs I identified from the episode transcript:

[
  {
    "label": "Josh's Controversial TV Appearance",
    "description": "Josh makes controversial remarks about Christian conservatives on a TV talk show, angering figures like Mary Marsh and Al Caldwell. This leads to a confrontation where Josh is forced to apologize, but the Christian group tries to extract concessions from the White House in exchange.",
    "characters": ["Josh", "Mary Marsh", "Al Caldwell", "Toby"],
    "themes": ["Religion in politics", "Free speech", "Compromise"]
  },
  {
    "label": "The President's Bicycle Accident",
    "description": "The President is injured in a bicycle accident, which becomes a source of embarrassment and distraction for the White House staff. They have to manage the PR fallout and the President's frustration over the incident.",
    "characters": ["President Bartlet", "Leo", "C.J."],
    "themes": ["Leadership", "Vulnerability", "Perception management"]
  },
  {

In [4]:
eval_copilot = EvaluationCopilot(openai_client, logging=True)
question = "You are good at extracting story arcs from transcript of TV episodes. You will be a given a transcript of an episode. Your task is to extract the main story arcs from this episode. A story arc is a narrative thread that runs through part or all of the episode, involving certain characters and exploring certain themes. An arc has a beginning, middle and end. It will often center around a conflict or challenge that the characters need to overcome. \n\nFor example, in an episode of Friends, one story arc might be:\n- Joey and Chandler centers around a humorous fight over a chair. The conflict begins when Chandler leaves the room for a moment and Joey takes his seat. When Chandler returns and demands the seat back, Joey refuses, leading to a series of tit-for-tat pranks between the two. The situation escalates comically when Joey decides to make a point about not sharing by wearing all of Chandler's clothes at once, declaring, \"Look at me, I'm Chandler. Could I BE wearing any more clothes?\" This is done as a form of exaggerated revenge for Chandler hiding Joey's underwear, and it leads to a memorable and funny moment involving physical comedy and slapstick humor.\n\nPlease carefully read through the transcript and identify the main story arcs. For each one, think about:\n<scratchpad>\n- What is the key conflict or challenge driving this arc? How is it set up and how is it resolved?\n- Which characters are most central to this arc? What role do they each play?\n- What deeper themes, if any, are explored through this storyline? \n</scratchpad>\n\nThen, output the story arcs in the following JSON format:\n\n<example>\n[\n  {\n    \"label\": \"Joey and Chandler's Fight\",\n    \"description\": \"Joey and Chandler bicker over a chair leading to escalating pranks including Joey wearing all of Chandler's clothes as an act of revenge.\",\n    \"characters\": [\"Joey\", \"Chandler\"],\n    \"themes\": [\"Friendship\", \"Conflict\", \"Petty revenge\"]\n  },\n  ...\n]\n</example>\n\n \n\nMake sure the story arc objects are printed in valid JSON format. Only include the valid JSON and do not include any other text. I just want the JSON from you.  "

question += f"\n\n<episode_transcript>\n{EPISODE_TRANSCRIPT}\n</episode_transcript>\n\n"

eval_input = EvaluationInput(question=question, answer=message)
eval_output = eval_copilot.evaluate(eval_input)
print(eval_output)

Prompt: 
    [System]
    Please act as an impartial judge and evaluate the quality of the response provided by an
    AI assistant to the user question displayed below. Your evaluation should consider factors
    such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of
    the response. Begin your evaluation by providing a short explanation. Be as objective and concise as
    possible using simple language. The explanation should be 2-3 bullet points that are easy to glance over. After providing your explanation, please rate the response on a scale of 1 to 5
    by strictly following this format: "[[explanation]],  [[rating]]", 
    for example: "Explanation:[[explanation..]], Rating: [[5]]".
    [Question]
    You are good at extracting story arcs from transcript of TV episodes. You will be a given a transcript of an episode. Your task is to extract the main story arcs from this episode. A story arc is a narrative thread that runs through part or all of

In [7]:
improvement_input = ImprovementInput(question=question, answer=message, score=eval_output.score, justification=eval_output.justification)
improvement_copilot = ImprovementCopilot(openai_client, logging=True)
improvement_output = improvement_copilot.suggest_improvements(improvement_input)

print(improvement_output)

Parse Improvement Response - Question Improvement: No suggestions; the user's question is clear and specific to the task of the AI.
Parse Improvement Response - Answer Improvement: The AI should exclude meta-conversation text surrounding the JSON formatted response, such as "Here are the main story arcs I identified from the episode transcript:" and "The End of Assistant’s Answer".
question_improvement="No suggestions; the user's question is clear and specific to the task of the AI." answer_improvement='The AI should exclude meta-conversation text surrounding the JSON formatted response, such as "Here are the main story arcs I identified from the episode transcript:" and "The End of Assistant’s Answer".'


In [8]:
improvement_output.answer_improvement

'The AI should exclude meta-conversation text surrounding the JSON formatted response, such as "Here are the main story arcs I identified from the episode transcript:" and "The End of Assistant’s Answer".'